# Redhift Distributions

from matplotlib import pyplot as plt
from sndata.des import sn3yr
from sndata.csp import dr3
from sndata.sdss import sako18

#### Table of Contents:
1. <a href='#combined'>Combined Surveys</a>: Redshift histogram for SDSS, DES, and CSP together.
1. <a href='#sdss'>SDSS</a>: Redshift histogram for the Sloan Digital Sky Survey
1. <a href='#csp'>CSP</a>: Redshift histogram for the Carnegie Supernova Project


In [ ]:
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt

from sndata.csp import dr3
from sndata.des import sn3yr
from sndata.sdss import sako18

# Output directory for figures
fig_dir = Path('./notebook_figs')
fig_dir.mkdir(exist_ok=True, parents=True)


## Combined Surveys <a id='combined'></a>

Redshift histogram for SDSS, DES, and CSP together.

In [ ]:
z_des = [d.meta['redshift'] for d in sn3yr.iter_data(True)]
z_csp = dr3.load_table(1)['z']

sdss_master = sako18.load_table('master')
z_sdss = sdss_master['zCMB']
z_sdss = z_sdss[z_sdss > 0]

In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(6, 6))
axis.hist(
    [z_sdss, z_csp, z_des], 
    label=['SDSS', 'CSP', 'DES'], 
    bins=np.arange(0, 1, .05),
    normed=True, 
    stacked=True
)

axis.set_xlim(0, 1)
axis.set_xlabel('Redshift', fontsize=12)
axis.set_ylabel('Number of Targets / Survey', fontsize=12)
axis.legend()

plt.show()

## SDSS <a id='sdss'></a>

Redshift histogram for the Sloan Digital Sky Survey


In [ ]:
sdss_df = sdss_master[sdss_master['zCMB'] >= 0].to_pandas(index='CID')
sdss_df.Classification.replace('SNIa?', 'SNIa', inplace=True)
sdss_df.Classification.replace('zSNIa', 'SNIa', inplace=True)
sdss_df.Classification.replace('SNIb', 'SNIb / c', inplace=True)
sdss_df.Classification.replace('SNIc', 'SNIb / c', inplace=True)
sdss_df.Classification.replace('SNIbc', 'SNIb / c', inplace=True)
sdss_df.Classification.replace('zSNIbc', 'SNIb / c', inplace=True)
sdss_df.Classification.replace('zSNII', 'SNII', inplace=True)
sdss_class_groups = sdss_df.groupby('Classification')

fig, axis = plt.subplots(1, 1, figsize=(6, 6))
ignore_types = ['Variable', 'AGN']
for class_name, class_data in sdss_class_groups:
    if class_name in ignore_types:
        continue

    axis.hist(list(class_data['zCMB']), label=class_name, bins=np.arange(0, 1, .05))

# z = (obsv_lambda / rest_lambda) - 1
u_eff = 3550
g_eff = 4680
r_eff = 6160
i_eff = 7480
z_eff = 8930

z_ug = (g_eff / u_eff) - 1
z_ur = (r_eff / u_eff) - 1
axis.axvline(z_ug, color='black', linestyle='--', label=r'u $\rightarrow$ g')
axis.axvline(z_ur, color='black', linestyle='-.', label=r'u $\rightarrow$ r')
    
axis.set_xlabel('Redshift', fontsize=12)
axis.set_ylabel('Number of Targets / Classification', fontsize=12)
plt.xlim(0, 1)
plt.tight_layout()
axis.legend()

plt.savefig(fig_dir / 'sdss_redshift.pdf')
plt.show()


## CSP <a id='csp'></a>

Redshift histogram for the carnegie supernova project


In [ ]:
table_1 = dr3.load_table(1).to_pandas(index='SN')
table_2 = dr3.load_table(2).to_pandas(index='SN')
csp_df = table_2.join(table_1)
csp_df.Subtype1.replace('normal', 'Normal', inplace=True)
csp_df.Subtype1.replace('---', 'Unknown', inplace=True)
csp_df.Subtype1.replace('06bt-like', 'Peculiar', inplace=True)
csp_df.Subtype1.replace('06gz-like', 'Peculiar', inplace=True)
csp_df.Subtype1.replace('86G-like', 'Peculiar', inplace=True)
csp_df.Subtype1.replace('91T-like', '91T', inplace=True)
csp_df.Subtype1.replace('91bg-like', '91bg', inplace=True)
csp_class_groups = csp_df.groupby('Subtype1')

fig, axis = plt.subplots(1, 1, figsize=(6, 6))
for class_name, class_data in csp_class_groups:
    axis.hist(list(class_data['z']), label=class_name, bins=np.arange(0, .1, .005))

axis.set_xlabel('Redshift', fontsize=12)
axis.set_ylabel('Number of Targets / Classification', fontsize=12)
axis.set_xlim(0, .1)
axis.legend()

plt.savefig(fig_dir / 'csp_redshift.pdf')
plt.show()
    